# Setup

In [1]:
import pandas as pd
import ccxt
import json
import time
from datetime import datetime
from datetime import timedelta
import plotly.offline as py
import plotly.graph_objs as go
import talib
py.init_notebook_mode(connected=True)

## Get OHLCV data from Binance

format ที่ได้จาก API

In [2]:
# [
#     [
#         1504541580000, // UTC timestamp in milliseconds, integer
#         4235.4,        // (O)pen price, float
#         4240.6,        // (H)ighest price, float
#         4230.0,        // (L)owest price, float
#         4230.7,        // (C)losing price, float
#         37.72941911    // (V)olume (in terms of the base currency), float
#     ],
#     ...
# ]

In [3]:
def datetimeFormat(x):
    return datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S')

เราอยากได้ข้อมูลมาเทสย้อนหลังสัก 100 วัน แต่ API ให้เราดึงได้ทีละ 1000 candlestick ดังนั้นต้องวนลูปดึงมาต่อกันทีละนิดๆ จนครบ จะได้

In [36]:
binance = ccxt.binance()

ayear_ago_milliseconds = int(round((datetime.now() - timedelta(minutes=1000)).timestamp()*1000))

ohlcv = binance.fetch_ohlcv('VEN/USDT', '1m', since=ayear_ago_milliseconds) # minute
# ohlcv = binance.fetch_ohlcv('VEN/BTC', '1m') # day
ohlcv_df = pd.DataFrame(ohlcv,columns = ['time', 'open', 'high', 'low', 'close', 'volumn'])

# change datetime format
ohlcv_df['time'] = ohlcv_df['time'].apply(datetimeFormat)

In [37]:
print(len(ohlcv_df))
ohlcv_df.head()

1000


,time,open,high,low,close,volumn
0,2018-07-14 18:55:00,1.9243,1.9243,1.9214,1.9214,1396.30
1,2018-07-14 18:56:00,1.9214,1.9221,1.9213,1.9213,21303.66
2,2018-07-14 18:57:00,1.9213,1.9215,1.9201,1.9215,7581.10
3,2018-07-14 18:58:00,1.9266,1.9266,1.9203,1.9214,1390.58
4,2018-07-14 18:59:00,1.9201,1.9201,1.9199,1.9199,3673.99


In [38]:
trace = go.Candlestick(
            x=ohlcv_df.time, 
            open=ohlcv_df.open, 
            high=ohlcv_df.high, 
            low=ohlcv_df.low, 
            close=ohlcv_df.close
        )

layout = go.Layout(xaxis = dict(rangeslider = dict(visible = True)))
data = [trace]

fig = go.Figure(data=data,layout=layout)
py.iplot(fig, filename='simple_candlestick')

> **Note that** the info from the last (current) candle may be incomplete until the candle is closed (until the next candle starts).

# Exponential Moving Average

**Ref :** http://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:moving_averages

In [39]:
# คำนวณจากราคา closing price

# ============= SMA ==============
# Daily Closing Prices: 11,12,13,14,15,16,17

# First day of 5-day SMA: (11 + 12 + 13 + 14 + 15) / 5 = 13

# Second day of 5-day SMA: (12 + 13 + 14 + 15 + 16) / 5 = 14

# Third day of 5-day SMA: (13 + 14 + 15 + 16 + 17) / 5 = 15

# ============= EMA ==============
# Initial SMA: 10-period sum / 10 

# Multiplier: (2 / (Time periods + 1) ) = (2 / (10 + 1) ) = 0.1818 (18.18%)

# EMA: {Close - EMA(previous day)} x multiplier + EMA(previous day). 

หาจำนวน 
- best shorterm 
- best longterm

In [40]:
ohlcv_df['shortEMA'] = talib.EMA(ohlcv_df['close'],timeperiod=10)
ohlcv_df['longEMA'] = talib.EMA(ohlcv_df['close'],timeperiod=20)

In [41]:
trace = go.Candlestick(
            x=ohlcv_df.time, 
            open=ohlcv_df.open, 
            high=ohlcv_df.high, 
            low=ohlcv_df.low, 
            close=ohlcv_df.close,
            opacity = 0.3
        )

short_EMA_line = go.Scatter(x=ohlcv_df.time, y=ohlcv_df.shortEMA, line = dict(color = '#27ae60'))
long_EMA_line = go.Scatter(x=ohlcv_df.time, y=ohlcv_df.longEMA, line = dict(color = '#2980b9'))

layout = go.Layout(xaxis = dict(rangeslider = dict(visible = True)))
data = [trace, short_EMA_line, long_EMA_line]

fig = go.Figure(data=data,layout=layout)
py.iplot(fig, filename='simple_candlestick')

# Backtest

ที่ทำให้ผล backtest ย้อนหลัง 70 และ ทำนายล่วงหน้า 30 วันดีที่สุด

In [49]:
# sell one ฿ for market price and receive $ right now
# print(exmo.id, exmo.create_market_sell_order('BTC/USD', 1))

# limit buy BTC/EUR, you pay €2500 and receive ฿1  when the order is closed
# print(exmo.id, exmo.create_limit_buy_order('BTC/EUR', 1, 2500.00))

# pass/redefine custom exchange-specific order params: type, amount, price, flags, etc...
# kraken.create_market_buy_order('BTC/USD', 1, {'trading_agreement': 'agree'})

In [104]:
24*60*100/1000

144.0

### Get my data

In [106]:
with open('privatekey.json') as json_data:
    privatekey = json.load(json_data)

In [103]:
binance = ccxt.binance({
    'apiKey': privatekey['api_key'],
    'secret': privatekey['api_secret'],
})

json_data = binance.fetch_balance()

balances = json_data['info']['balances']
balances_df = pd.DataFrame(balances).sort_values(by=['free'],ascending=False)

In [105]:
balances_df[0:10]

,asset,free,locked
22,ICN,64.93500000,0.00000000
74,VEN,50.88000000,0.00000000
46,REP,2.36563200,0.00000000
41,IOTA,130.39548000,0.00000000
65,BAT,0.48500000,0.00000000
0,BTC,0.01291322,0.00000000
16,USDT,0.00868727,0.00000000
58,ETC,0.00304000,0.00000000
1,LTC,0.00027743,0.00000000
2,ETH,0.00006516,0.00000000


free คือ ส่วนที่เก็บไว้ไม่ได้ทำไรเลย lock คือส่วนที่ตั้งขายไว้อยู่แต่ยังขายไม่ได้ ถ้าจะถามว่าเรามีเหรียญทั้งหมดเท่าไหร่ ควรคิดโดนรวมทั้งสอง column